In [8]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm_notebook

def timedelta_to_hhmmss(x):
    hh = str(x.components.hours + (24 * x.components.days)).zfill(2)
    mm = str(x.components.minutes).zfill(2)
    ss = str(x.components.seconds).zfill(2)
    return hh + ':' + mm + ':' + ss

def get_f_node(links, vlinks, mode):
    f_nodes = []
    i = 1 if links['LINK_ID'].iloc[0] == '0' else 0
    
#     print(len(set(vlinks[vlinks['LINK_ID'] == links.LINK_ID.unique()[i]])))
    for _, row in vlinks[vlinks['LINK_ID'] == links.LINK_ID.unique()[i]].iterrows():
        f_nodes.append(row['F_NODE'])
        if mode == 1:
            f_nodes.append(row['T_NODE'])
        
#     for _, row in vlinks[vlinks['LINK_ID'] == links.LINK_ID.unique()[i+1]].iterrows():
#         f_nodes.append(row['F_NODE'])
#     print(Counter(f_nodes))
        
    for _, row in vlinks[vlinks['LINK_ID'] == links.LINK_ID.unique()[i+1]].iterrows():
        if row['F_NODE'] in f_nodes:
            f_nodes.remove(row['F_NODE'])
        if row['T_NODE'] in f_nodes:
            f_nodes.remove(row['T_NODE'])
    
#     print(Counter(f_nodes))
    if len(vlinks[vlinks['LINK_ID'] == links.LINK_ID.unique()[i]].LINK_NO.unique()) != 1 and len(set(f_nodes)) <= 1:
        return np.nan, np.nan
    
    min_cnt = 0
    for f_node in f_nodes:
        if f_nodes.count(f_node) == min(Counter(f_nodes).values()):
            min_cnt = min_cnt + 1
    
#     print(min_cnt)
    if min_cnt != min(Counter(f_nodes).values()):
#     if min_cnt > 1:
        return np.nan, np.nan
    
    for f_node in f_nodes:
        if f_nodes.count(f_node) == min(Counter(f_nodes).values()):
            t_node = vlinks[(vlinks['LINK_ID'] == links.LINK_ID.unique()[i]) & (vlinks['F_NODE'] == f_node)]['T_NODE'].item()
            return f_node, t_node        
    return np.nan, np.nan

In [2]:
# file_name = 'input/네트워크10(200319,v2,M6439수정,김민기).xlsx'
file_name = 'input/네트워크_개선6(200416,v5,노선오류수정,김민기).xlsx'
df_visum_link = pd.read_excel(file_name, sheet_name='Visum_link', dtype={'F_ID': str, 'LINK_ID': str, 'T_ID': str})
df_visum_stop = pd.read_excel(file_name, sheet_name='Visum_stop')
df_time_item = pd.read_excel(file_name, sheet_name='Time', dtype={'노선ID': str, 'ISC노선ID': str, '정류소순번': str, '정류소ID': str, 'VISUM정류소NO': str, 'ARR': str, 'DEP': str})
df_link_id = pd.read_excel(file_name, sheet_name='Link_ID', dtype={'ROUTE_ID': str, 'LINK_ID': str})
df_route_info = pd.read_excel(file_name, sheet_name='Route', dtype={'노선ID': str, 'ISC노선ID': str, '기점첫차시간': str, '배차간격': str})

In [10]:
trace = False
bus_op = '인가운행횟수'

df_lri_columns = ['$LINEROUTEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'INDEX', 'ISROUTEPOINT', 'NODENO', 'STOPPOINTNO', 'POSTLENGTH', 'ADDVAL']
df_line_route_items = pd.DataFrame(columns=df_lri_columns)
df_line_route_items.to_csv('output/line_route_items.csv', index=False, encoding='utf-8-sig')

# df_tpi_columns = ['$TIMEPROFILEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'TIMEPROFILENAME', 'INDEX', 'LRITEMINDEX', 'ALIGHT', 'BOARD', 'ARR', 'DEP', 'NUMFAREPOINTS', 'NUMFAREPOINTSBOARD', 'NUMFAREPOINTSTHROUGH', 'NUMFAREPOINTSALIGHT', 'ADDVAL', '행번호', '정류장순서']
df_tpi_columns = ['$TIMEPROFILEITEM:LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'TIMEPROFILENAME', 'INDEX', 'LRITEMINDEX', 'ALIGHT', 'BOARD', 'ARR', 'DEP', 'NUMFAREPOINTS', 'NUMFAREPOINTSBOARD', 'NUMFAREPOINTSTHROUGH', 'NUMFAREPOINTSALIGHT', 'ADDVAL']
df_time_profile_items = pd.DataFrame(columns=df_tpi_columns)
df_time_profile_items.to_csv('output/time_profile_items.csv', index=False, encoding='utf-8-sig')

df_vji_columns = ['$VEHJOURNEY:NO', 'NAME', 'DEP', 'LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE', 'TIMEPROFILENAME', 'FROMTPROFITEMINDEX', 'TOTPROFITEMINDEX', 'OPERATORNO', 'ADDVAL1', 'ADDVAL2', 'ADDVAL3', 'SERVTRIPPATNO']
df_vehicle_journey_items = pd.DataFrame(columns=df_vji_columns)
df_vehicle_journey_items.to_csv('output/vehicle_journeys.csv', index=False, encoding='utf-8-sig')

df_vjs_columns = ['$VEHJOURNEYSECTION:VEHJOURNEYNO', 'NO', 'FROMTPROFITEMINDEX', 'TOTPROFITEMINDEX', 'VALIDDAYSNO', 'VEHCOMBNO', 'VEHCOMBSET', 'ISOPTIONALREINFORCEMENT', 'PREPREPTIME', 'USESPECPREPREPTIME', 'POSTPREPTIME', 'USESPECPOSTPREPTIME', 'OPERATINGPERIODNO']
df_vehicle_journey_sections = pd.DataFrame(columns=df_vjs_columns)
df_vehicle_journey_sections.to_csv('output/vehicle_journey_sections.csv', index=False, encoding='utf-8-sig')

MAX_INDEX = {}

DIRECTIONCODE = '>'
POSTLENGTH = '0km'
ADDVAL = 0

ARR = '00:00:00'
DEP = '00:00:00'

INDEX = 1; SINDEX = 1; N_CNT = 0; S_CNT = 0; JINDEX = 1
F_NODE = np.nan; T_NODE = np.nan; prev_route_id = np.nan; prev_stop = np.nan; prev_link = np.nan; prev_link_no = np.nan; prev_route_name = np.nan

df_link = df_link_id
# df_link = df_link_id[df_link_id['V9'] == 1]
# df_link = df_link_id[(df_link_id['ROUTE_NAME'] == '1(강화)') | (df_link_id['ROUTE_NAME'] == '70') | (df_link_id['ROUTE_NAME'] == '60-2') | (df_link_id['ROUTE_NAME'] == '6(강화)') | (df_link_id['ROUTE_NAME'] == '3(강화)') | (df_link_id['ROUTE_NAME'] == '4(강화)') | (df_link_id['ROUTE_NAME'] == '10(강화)') | (df_link_id['ROUTE_NAME'] == '11(강화)') | (df_link_id['ROUTE_NAME'] == '12(강화)') | (df_link_id['ROUTE_NAME'] == '13(강화)') | (df_link_id['ROUTE_NAME'] == '14(강화)') | (df_link_id['ROUTE_NAME'] == '15(강화)') | (df_link_id['ROUTE_NAME'] == '17(강화)') | (df_link_id['ROUTE_NAME'] == '20(강화)') | (df_link_id['ROUTE_NAME'] == '21(강화)') | (df_link_id['ROUTE_NAME'] == '23(강화)') | (df_link_id['ROUTE_NAME'] == '24(강화)') | (df_link_id['ROUTE_NAME'] == '25(강화)') | (df_link_id['ROUTE_NAME'] == '26(강화)') | (df_link_id['ROUTE_NAME'] == '27(강화)') | (df_link_id['ROUTE_NAME'] == '30(강화)') | (df_link_id['ROUTE_NAME'] == '31A(강화)') | (df_link_id['ROUTE_NAME'] == '32(강화)') | (df_link_id['ROUTE_NAME'] == '33(강화)') | (df_link_id['ROUTE_NAME'] == '34(강화)') | (df_link_id['ROUTE_NAME'] == '49(강화)') | (df_link_id['ROUTE_NAME'] == '37(강화)') | (df_link_id['ROUTE_NAME'] == '40(강화)') | (df_link_id['ROUTE_NAME'] == '41(강화)') | (df_link_id['ROUTE_NAME'] == '42(강화)') | (df_link_id['ROUTE_NAME'] == '43(강화)') | (df_link_id['ROUTE_NAME'] == '44(강화)') | (df_link_id['ROUTE_NAME'] == '45(강화)') | (df_link_id['ROUTE_NAME'] == '48(강화)') | (df_link_id['ROUTE_NAME'] == '50(강화)') | (df_link_id['ROUTE_NAME'] == '51(강화)') | (df_link_id['ROUTE_NAME'] == '52(강화)') | (df_link_id['ROUTE_NAME'] == '53(강화)') | (df_link_id['ROUTE_NAME'] == '54(강화)') | (df_link_id['ROUTE_NAME'] == '55(강화)') | (df_link_id['ROUTE_NAME'] == '61A(강화)') | (df_link_id['ROUTE_NAME'] == '38A(강화)') | (df_link_id['ROUTE_NAME'] == '57(강화)') | (df_link_id['ROUTE_NAME'] == '902(강화)') | (df_link_id['ROUTE_NAME'] == '901(강화)') | (df_link_id['ROUTE_NAME'] == '800') | (df_link_id['ROUTE_NAME'] == '700-1') | (df_link_id['ROUTE_NAME'] == '22(강화)') | (df_link_id['ROUTE_NAME'] == '60-5') | (df_link_id['ROUTE_NAME'] == '18(강화)') | (df_link_id['ROUTE_NAME'] == '2(강화)') | (df_link_id['ROUTE_NAME'] == '5(강화)') | (df_link_id['ROUTE_NAME'] == '7A(강화)') | (df_link_id['ROUTE_NAME'] == '7B(강화)') | (df_link_id['ROUTE_NAME'] == '8A(강화)') | (df_link_id['ROUTE_NAME'] == '8B(강화)') | (df_link_id['ROUTE_NAME'] == '16-1(강화)') | (df_link_id['ROUTE_NAME'] == '16-2(강화)') | (df_link_id['ROUTE_NAME'] == '36(강화)') | (df_link_id['ROUTE_NAME'] == '58(강화)') | (df_link_id['ROUTE_NAME'] == '59(강화)') | (df_link_id['ROUTE_NAME'] == '10A(강화)') | (df_link_id['ROUTE_NAME'] == '12A(강화)') | (df_link_id['ROUTE_NAME'] == '30A(강화)') | (df_link_id['ROUTE_NAME'] == '53A(강화)') | (df_link_id['ROUTE_NAME'] == '61B(강화)') | (df_link_id['ROUTE_NAME'] == '11A(강화)') | (df_link_id['ROUTE_NAME'] == '31B(강화)') | (df_link_id['ROUTE_NAME'] == '38B(강화)') | (df_link_id['ROUTE_NAME'] == '9(강화)') | (df_link_id['ROUTE_NAME'] == '35A(강화)') | (df_link_id['ROUTE_NAME'] == '35B(강화)') | (df_link_id['ROUTE_NAME'] == '44A(강화)') | (df_link_id['ROUTE_NAME'] == '19(강화)') | (df_link_id['ROUTE_NAME'] == '23A(강화)') | (df_link_id['ROUTE_NAME'] == '39(강화)') | (df_link_id['ROUTE_NAME'] == '39A(강화)') | (df_link_id['ROUTE_NAME'] == '904(강화)') | (df_link_id['ROUTE_NAME'] == '903(강화)') | (df_link_id['ROUTE_NAME'] == '905(강화)') | (df_link_id['ROUTE_NAME'] == '906(강화)')]
# df_link = df_link_id[df_link_id['ROUTE_NAME'] == '205_변경']
# df_link = df_link_id[(df_link_id['ROUTE_NAME'] == '62') | (df_link_id['ROUTE_NAME'] == '3(강화)_변경')]
for _, l in tqdm_notebook(df_link.iterrows(), total=df_link.shape[0]):
    route_id = l['ROUTE_ID']; route_name = l['ROUTE_NAME']; link_id = l['LINK_ID']; link_idx = l['INDEX']
    if link_id == '0':
        continue

    if prev_route_id != route_id:
        for i, item in df_line_route_items.iloc[::-1].iterrows():
            if item['ISROUTEPOINT'] == 1:
                break;
            df_line_route_items = df_line_route_items.drop(i, 0)
        
        for i, item in df_time_profile_items.iloc[::-1].iterrows():
            if item['ALIGHT'] == 0 and item['BOARD'] == 0:
                df_time_profile_items = df_time_profile_items.drop(i, 0)
            if item['ALIGHT'] == 1 and item['BOARD'] == 1:
                df_time_profile_items.loc[i, 'BOARD'] = 0
                break;
        
        if SINDEX > 1:
            df_route = df_route_info[df_route_info['ISC노선ID'] == prev_route_id]
            if len(df_route) != 0:
                for i in range(1, int(df_route[bus_op]) + 1):
                    dep_time = timedelta_to_hhmmss(pd.to_timedelta(df_route['기점첫차시간'].item()) + (pd.to_timedelta(df_route['배차간격'].item()) * i))
                    df_vehicle_journey_items = df_vehicle_journey_items.append(pd.Series([JINDEX, '', dep_time, prev_route_id, prev_route_name, DIRECTIONCODE, 1, 1, MAX_INDEX[prev_route_id], '', 0, 0, 0, 0], index=df_vji_columns), ignore_index=True)
                    df_vehicle_journey_sections = df_vehicle_journey_sections.append(pd.Series([JINDEX, 1, 1, MAX_INDEX[prev_route_id], 1, '', '', 0, '0s', 0, '0s', 0, ''], index=df_vjs_columns), ignore_index=True)
                    JINDEX = JINDEX + 1
                    
        df_line_route_items.to_csv('output/line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_line_route_items = pd.DataFrame(columns=df_lri_columns) 
        
        df_time_profile_items.to_csv('output/time_profile_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_time_profile_items = pd.DataFrame(columns=df_tpi_columns)
        
        df_vehicle_journey_items.to_csv('output/vehicle_journeys.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_vehicle_journey_items = pd.DataFrame(columns=df_vji_columns)
        
        df_vehicle_journey_sections.to_csv('output/vehicle_journey_sections.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
        df_vehicle_journey_sections = pd.DataFrame(columns=df_vjs_columns)
        
        INDEX = 1; SINDEX = 1; N_CNT = 0; S_CNT = 0;
        
        F_NODE, T_NODE = get_f_node(df_link[df_link['ROUTE_ID'] == route_id], df_visum_link, 0)
        if np.isnan(F_NODE) or np.isnan(T_NODE):
            F_NODE, T_NODE = get_f_node(df_link[df_link['ROUTE_ID'] == route_id], df_visum_link, 1)
#         print(F_NODE, T_NODE)
        
        if np.isnan(F_NODE):
            if link_id == '1690012500':
                F_NODE, T_NODE = 377, 7297
            elif link_id == '1690116600':
                F_NODE, T_NODE = 68, 7206
            elif link_id == '1690083700':
                F_NODE, T_NODE = 25, 7148
            elif link_id == '1650123400':
                F_NODE, T_NODE = 2455, 2475
            elif link_id == '1640070400':
                F_NODE, T_NODE = 1033, 3608
            elif link_id == '1610184600':
                F_NODE, T_NODE = 804, 9533
            elif link_id == '1640135500':
                F_NODE, T_NODE = 1537, 4505
            elif link_id == '1630055900':
                F_NODE, T_NODE = 1405, 11988
            elif link_id == '1650106300':
                F_NODE, T_NODE = 2894, 11231
            elif link_id == '2100088600':
                F_NODE, T_NODE = 2673, 11102
            elif link_id == '1660103400':
                F_NODE, T_NODE = 2778, 11109
            elif link_id == '1680189000':
                F_NODE, T_NODE = 11630, 5228
            elif link_id == '1640090500': #91-1(순환)_신설
                F_NODE, T_NODE = 3338, 764
            elif link_id == '1660097100': #103-1_변경
                F_NODE, T_NODE = 5226, 1823
            elif link_id == '1660051500': #592-1_신설
                F_NODE, T_NODE = 1909, 3990
            else:
                print(route_name)
        
        prev_stop = np.nan; prev_link = np.nan; prev_link_no = np.nan;
        df_time = df_time_item[df_time_item['ISC노선ID'] == route_id]
        df_time_len = len(df_time)
        if '개편유형' in df_route_info.columns:
            if (df_route_info[df_route_info['ISC노선ID'] == route_id]["개편유형"] != "존치").item():
                df_time_len = 0
        
        ARR = '00:00:00'
        DEP = '00:00:00'
    prev_route_id = route_id
    prev_route_name = route_name
    
    if prev_link == link_id:
        continue
    prev_link = link_id
    
    link_set = df_visum_link[df_visum_link['LINK_ID'] == link_id]
#     print(len(link_set))
    while len(link_set) > 0:
        if np.isnan(T_NODE):
            df_vlink = link_set[link_set['F_NODE'] == F_NODE]
        else:
            df_vlink = link_set[(link_set['F_NODE'] == F_NODE) & (link_set['T_NODE'] == T_NODE)]
        if len(df_vlink) == 0:
            break
        elif len(df_vlink) > 1:
            if (df_vlink['LINK_NO'] == 4873).any():
                df_vlink = df_vlink[df_vlink['LINK_NO'] == 4873]
                link_set = link_set.drop(df_vlink.index)
            else:
                print(df_line_route_items)
                print(df_vlink)
                lno = int(input('LINK_NO: '))
                df_vlink = df_vlink[df_vlink['LINK_NO'] == lno]
                link_set = link_set.drop(df_vlink.index)
        else:
            link_set = link_set.drop(link_set[(link_set['F_NODE'] == F_NODE) | ((link_set['T_NODE'] == F_NODE))].index)
        
        if np.isnan(T_NODE):
            T_NODE = df_vlink['T_NODE'].item()
            
        for _, vl in df_vlink.iterrows():
            link_no = vl['LINK_NO']
            if prev_link_no == link_no:
                continue
            prev_link_no = link_no
            
            df_vstop = (df_visum_stop[(df_visum_stop['LINK_NO'] == link_no) & (df_visum_stop['F_NODE'] == F_NODE)]).sort_values(by='INDEX')
            for _, vs in df_vstop.iterrows():
                if prev_stop == vs['STOP_NO']:
                    continue
                prev_stop = vs['STOP_NO']
                    
                stop_info = df_time[(df_time['VISUM정류소NO'] == str(vs['STOP_NO'])) & (df_time['정류소순번'] == str(SINDEX))].head(1)
#                 if len(stop_info) == 0:
#                     stop_info = df_time[df_time['정류소순번'] == str(SINDEX)].head(1)

                if df_time_len == 0:
                    if SINDEX == 1:
                        df_time_profile_items = df_time_profile_items.append(pd.Series([route_id, route_name, '>', 1, SINDEX, INDEX, 0, 1, '00:00:00', '00:00:00', 0, 0, 0, 0, 0], index=df_tpi_columns), ignore_index=True)
                    else:
                        df_time_profile_items = df_time_profile_items.append(pd.Series([route_id, route_name, '>', 1, SINDEX, INDEX, 1, 1, '00:00:00', '00:00:00', 0, 0, 0, 0, 0], index=df_tpi_columns), ignore_index=True)
                    MAX_INDEX[route_id] = SINDEX
                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_lri_columns), ignore_index=True)
                    INDEX = INDEX + 1 
                    SINDEX = SINDEX + 1
                    S_CNT = S_CNT + 1
                elif len(stop_info) != 0:
                    if SINDEX == 1:
                        df_time_profile_items = df_time_profile_items.append(pd.Series([route_id, route_name, '>', 1, SINDEX, INDEX, 0, 1, stop_info['ARR'].item(), stop_info['DEP'].item(), 0, 0, 0, 0, 0], index=df_tpi_columns), ignore_index=True)
                    else:
                        df_time_profile_items = df_time_profile_items.append(pd.Series([route_id, route_name, '>', 1, SINDEX, INDEX, 1, 1, stop_info['ARR'].item(), stop_info['DEP'].item(), 0, 0, 0, 0, 0], index=df_tpi_columns), ignore_index=True)
                    MAX_INDEX[route_id] = SINDEX
                    ARR = stop_info['DEP'].item()
                    DEP = ARR
                    df_time = df_time.drop(stop_info.index)
                    df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 1, '', vs['STOP_NO'], POSTLENGTH, ADDVAL], index=df_lri_columns), ignore_index=True)
                    INDEX = INDEX + 1 
                    SINDEX = SINDEX + 1
                    S_CNT = S_CNT + 1

            trace and print("[", link_idx, "]", link_id, "( ", link_no, "):", F_NODE, '→', T_NODE)
            if len(link_set) == 0:
                F_NODE = T_NODE
                T_NODE = np.nan
            else:
                TEMP = link_set[(link_set['F_NODE'] == T_NODE) & (link_set['T_NODE'] != F_NODE)].T_NODE
                F_NODE = T_NODE
                T_NODE = TEMP.item()
            if SINDEX > 1:
                df_line_route_items = df_line_route_items.append(pd.Series([route_id, route_name, DIRECTIONCODE, INDEX, 0, F_NODE, '', POSTLENGTH, ADDVAL], index=df_lri_columns), ignore_index=True)
                INDEX = INDEX + 1
                N_CNT = N_CNT + 1
            prev_stop = np.nan
                
for i, item in df_line_route_items.iloc[::-1].iterrows():
    if item['ISROUTEPOINT'] == 1:
        break;
    df_line_route_items = df_line_route_items.drop(i, 0)

for i, item in df_time_profile_items.iloc[::-1].iterrows():
    if item['ALIGHT'] == 0 and item['BOARD'] == 0:
        df_time_profile_items = df_time_profile_items.drop(i, 0)
    if item['ALIGHT'] == 1 and item['BOARD'] == 1:
        df_time_profile_items.loc[i, 'BOARD'] = 0
        break;
        
if SINDEX > 1:
    df_route = df_route_info[df_route_info['ISC노선ID'] == prev_route_id]
    if len(df_route) != 0:
        for i in range(1, int(df_route[bus_op]) + 1):
            dep_time = timedelta_to_hhmmss(pd.to_timedelta(df_route['기점첫차시간'].item()) + (pd.to_timedelta(df_route['배차간격'].item()) * i))
            df_vehicle_journey_items = df_vehicle_journey_items.append(pd.Series([JINDEX, '', dep_time, prev_route_id, prev_route_name, DIRECTIONCODE, 1, 1, MAX_INDEX[prev_route_id], '', 0, 0, 0, 0], index=df_vji_columns), ignore_index=True)
            df_vehicle_journey_sections = df_vehicle_journey_sections.append(pd.Series([JINDEX, 1, 1, MAX_INDEX[prev_route_id], 1, '', '', 0, '0s', 0, '0s', 0, ''], index=df_vjs_columns), ignore_index=True)
            JINDEX = JINDEX + 1    

if len(df_line_route_items) == 0:
    print(route_name)
df_line_route_items.to_csv('output/line_route_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
df_time_profile_items.to_csv('output/time_profile_items.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
df_vehicle_journey_items.to_csv('output/vehicle_journeys.csv', mode='a', index=False, encoding='utf-8-sig', header=False)
df_vehicle_journey_sections.to_csv('output/vehicle_journey_sections.csv', mode='a', index=False, encoding='utf-8-sig', header=False)